### Nama : Anandita Agung Nugraha
### NIM  : 2209106042
##### Link Dataset : https://www.kaggle.com/datasets/muratkokludataset/rice-image-dataset/code 

### **Import Library**

In [3]:
import tensorflow as tf
import pathlib
import splitfolders
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
import os


### Membuat Folder Train, Test , Validation

In [4]:
import pathlib
Dataset_dir = "Rice_Image_Dataset"
Dataset_dir =pathlib.Path(Dataset_dir)

splitfolders.ratio(Dataset_dir, output="Rice_dataset", seed=101, ratio=(.7, .2, .1))

In [5]:
train_img_path = 'Rice_dataset/train'
test_img_path = 'Rice_dataset/val'
validation_img_path = 'Rice_dataset/test'

### Melakukan Image Augmentation dan Membuat Arsitektur CNN

In [6]:
train_datagen = ImageDataGenerator(
     rescale=1./255,          
    rotation_range=40,        
    width_shift_range=0.2,    
    height_shift_range=0.2, 
    shear_range=0.2,          
    zoom_range=0.2,           
    horizontal_flip=True,     
    fill_mode='nearest'
)


train_generator = train_datagen.flow_from_directory(
    train_img_path,
    target_size=(224, 224),  
    batch_size=32,
    shuffle = True,
    class_mode='categorical'
)

val_datagen = ImageDataGenerator(
    rescale=1./255
    )
val_generator = val_datagen.flow_from_directory(
    validation_img_path,
    target_size=(224,224), 
    batch_size=32,
    shuffle = True,
    class_mode='categorical'
)
test_datagen = ImageDataGenerator(
    rescale=1./255
    )
test_generator = test_datagen.flow_from_directory(
    test_img_path,
    target_size=(224,224), 
    batch_size=32,
    shuffle = True,
    class_mode='categorical'
)

Found 52500 images belonging to 5 classes.
Found 7500 images belonging to 5 classes.
Found 15000 images belonging to 5 classes.


### Menampilkan nama Kelas yang ada pada Dataset

In [7]:
class_names = list(train_generator.class_indices.keys())
class_names

['Arborio', 'Basmati', 'Ipsala', 'Jasmine', 'Karacadag']

### Modelling

### Membuat pre-trained model

In [8]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False



### Membuat Model Sequential

In [9]:
model = Sequential()

model.add(base_model)
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))


### Model Compile

In [10]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

### Proses Training menggunakan fungsi Callback

In [12]:

early_stopping = keras.callbacks.EarlyStopping( monitor='val_accuracy',patience=5,restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  
    factor=0.5,   
    patience=4,         
    min_lr=1e-6          
)

history = model.fit(
    train_generator,
    epochs=1,
    validation_data=val_generator,
    callbacks= [early_stopping, reduce_lr]
)

 198/1641 [==>...........................] - ETA: 1:10:01 - loss: 0.1547 - accuracy: 0.9530

 ### Evaluasi Model

In [ ]:
model.evaluate(val_generator)

235/235 [==============================] - 404s 2s/step - loss: 0.2367 - accuracy: 0.9188


[0.2366613894701004, 0.9187999963760376]

### Plot akurasi dan loss pada training dan validation

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(history.history["accuracy"],label = "Train Accuracy", color = "red")
plt.plot(history.history["val_accuracy"],label = "Validation Accuracy", color = "green")
plt.title("Model Accuracy", color = "blue", size = 13)
plt.legend()
plt.show()

NameError: name 'history' is not defined

<Figure size 500x500 with 0 Axes>

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(history.history["accuracy"],label = "Train Accuracy", color = "red")
plt.plot(history.history["val_accuracy"],label = "Validation Accuracy", color = "green")
plt.title("Model Accuracy", color = "blue", size = 13)
plt.legend()
plt.show()